### Imports

In [1]:
# import usual packages
import json
import nltk
from nltk.collocations import *
import datetime as dt
import locale
import spacy
from tqdm import tqdm
import pprint
import pandas as pd
from ast import literal_eval
import os

### Read Data

In [3]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#change cwd
%cd drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [5]:
# paths
FILTERED_PATH="filtered_4_26"

In [ ]:
# import spacy package
# !python -m spacy download de_core_news_lg

#### Read Data

In [8]:
dfs = []

# loop through files 
for filename in os.listdir(FILTERED_PATH):
    # if csv file, load and add to dfs  
    if filename.endswith(".csv"):
        file_path = os.path.join(FILTERED_PATH, filename)
        df = pd.read_csv(file_path, index_col=0, converters={"text":literal_eval})
        dfs.append(df)

# combine files in df
df = pd.concat(dfs, ignore_index=True)

In [9]:
df.shape

(63359, 8)

# Preprocessing 

### Main steps:

- load (german) spacy model 
- convert corpus to spacy language object
- run preprocessing function to clean corpus

- review stopwords to keep causal language/verbs/pronouns

- create two versions of corpus for comparison: 
    - fully cleaned w/o entities/stopwords/proper nouns
    - partially cleaned w/o entities/proper nouns but w/causal stopwords


In [10]:
# custom module
import preprocessing

In [13]:
# Load model
spacy_mod = spacy.load("de_core_news_lg", disable=['ner', 'parser', 'tagger'])

In [14]:
# read custom stopwords

# open list of custom stopwords
custom_stop_words= open("custom_stopwords.txt").read().split()

# add custom stopwords to model
for word in custom_stop_words:
  spacy_mod.Defaults.stop_words.add(word)

In [15]:
smaller_df=df.iloc[:10000,:]

In [16]:
# convert corpus to language object
spacy_lang = []
for i, doc in tqdm(smaller_df.iterrows()): 
  spacy_lang.append(spacy_mod("".join(doc['text'])))

10000it [06:16, 26.55it/s]


In [17]:
# preprocess: remove stopwords
spacy_cleaned = []
for doc in tqdm(spacy_lang): 
    spacy_cleaned.append(preprocessing.preprocess(doc, remove_ent=True))

100%|██████████| 10000/10000 [00:05<00:00, 1897.11it/s]


In [18]:
# compare dirty and cleaned corpus
print(spacy_lang[0][0:30], len(spacy_lang[0]), sep='\n')
print(spacy_cleaned[0][0:10], len(spacy_cleaned[0]), sep='\n')

NOTRUFFeuerwehr/Lebensrettung Tel. 112Polizei Tel. 110KrankentransportTel. 03 85/500 02 17APOTHEKEN-NOTDIENSTSo 8 Uhr bis Di 8 UhrKlütz: Klützer Apotheke, Am Markt 2Tel. 03 88
441
['notruffeuerwehr', 'lebensrettung', 'uhrklütz', 'klützer', 'apotheke', 'markt', 'uhrmo', 'uhrtel', 'ratsapotheke', 'markt']
32


In [19]:
spacy_lang=[]

# Collocation Analysis

## Main steps:

- Create bi/tri/quad- ngrams
- Rank order ngrams by raw frequency (print to txt files)

- Examine association measures for top 50 ngrams
- Analyse differences in ngrams across documents
- Check for collocation strength/significance testing

- plot comparisons: association strength (dot chart) / network graphs / biplots (using semantic similarity)

## Methods:
- rank_bigrams: ranks bigrams by given metric 
- rank_trigrams: ranks trigrams by given metric 
- rank_quadgrams: ranks quadgrams by given metric 
- ngram_comparison: compares ngram by different metrics

In [20]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics.association import BigramAssocMeasures

from nltk.collocations import TrigramCollocationFinder
from nltk.metrics.association import TrigramAssocMeasures

from nltk.collocations import QuadgramCollocationFinder
from nltk.metrics.association import QuadgramAssocMeasures


In [21]:
# rank bigrams by given metric 
def rank_bigrams(corpus, metric, from_words=False, threshold=int, path=None):
    """
    Find and rank ngrams from the supplied corpus using the given
    association metric. Write the trigrams out to the given path if
    supplied otherwise return the list in memory.
    
    Args:
    from_words (bool, optional): If True, unlist all documents and create ngrams from words.
    threshold (int): ignore all ngrams which occur less than n times in the corpus
    path (str): path to save ngram list as .txt file
    
    """
    
    if from_words == True:
        #turn all doc tokens into one single list
        unlist_corpus = [item for items in corpus for item in items]
        # Create a collocation ranking utility from corpus
        finder = BigramCollocationFinder.from_words(unlist_corpus)
        # Apply frequency filter = at least 3 times
        finder.apply_freq_filter(threshold)
        # Rank collocations by an association metric
        scored = finder.score_ngrams(metric)
            
    else: 
        finder = BigramCollocationFinder.from_documents(corpus)
        finder.apply_freq_filter(threshold)
        scored = finder.score_ngrams(metric)

    if path:
    # Write to disk as tab-delimited file
        with open(path, 'w') as f:
            f.write("Collocation\tScore ({})\n".format(metric.__name__))
            for ngram, score in scored:
                f.write("{}\t{}\n".format(repr(ngram), score))
    else:
        return scored

In [22]:
#ranks trigrams by given metric
def rank_trigrams(corpus, metric, from_words=False, threshold=int, path=None):
    """
    Find and rank ngrams from the supplied corpus using the given
    association metric. Write the trigrams out to the given path if
    supplied otherwise return the list in memory.
    
    Args:
    from_words (bool, optional): If True, unlist all documents and create ngrams from words.
    threshold (int): ignore all ngrams which occur less than n times in the corpus
    path (str): path to save ngram list as .txt file
    
    """
    
    if from_words == True:
        #turn all doc tokens into one single list
        unlist_corpus = [item for items in corpus for item in items]
        # Create a collocation ranking utility from corpus
        finder = TrigramCollocationFinder.from_words(unlist_corpus)
        # Apply frequency filter = at least 3 times
        finder.apply_freq_filter(threshold)
        # Rank collocations by an association metric
        scored = finder.score_ngrams(metric)
            
    else: 
        finder = TrigramCollocationFinder.from_documents(corpus)
        finder.apply_freq_filter(threshold)
        scored = finder.score_ngrams(metric)

    if path:
    # Write to disk as tab-delimited file
        with open(path, 'w') as f:
            f.write("Collocation\tScore ({})\n".format(metric.__name__))
            for ngram, score in scored:
                f.write("{}\t{}\n".format(repr(ngram), score))
    else:
        return scored
    

In [23]:
# rank quadgrams by given metric 
def rank_quadgrams(corpus, metric, from_words=False, threshold=int, path=None):
    """
    Find and rank ngrams from the supplied corpus using the given
    association metric. Write the trigrams out to the given path if
    supplied otherwise return the list in memory.
    
    Args:
    from_words (bool, optional): If True, unlist all documents and create ngrams from words.
    threshold (int): ignore all ngrams which occur less than n times in the corpus
    path (str): path to save ngram list as .txt file
    
    """
    
    if from_words == True:
        #turn all doc tokens into one single list
        unlist_corpus = [item for items in corpus for item in items]
        # Create a collocation ranking utility from corpus
        finder = QuadgramCollocationFinder.from_words(unlist_corpus)
        # Apply frequency filter = at least 3 times
        finder.apply_freq_filter(threshold)
        # Rank collocations by an association metric
        scored = finder.score_ngrams(metric)
            
    else: 
        finder = QuadgramCollocationFinder.from_documents(corpus)
        finder.apply_freq_filter(threshold)
        scored = finder.score_ngrams(metric)

    if path:
    # Write to disk as tab-delimited file
        with open(path, 'w') as f:
            f.write("Collocation\tScore ({})\n".format(metric.__name__))
            for ngram, score in scored:
                f.write("{}\t{}\n".format(repr(ngram), score))
    else:
        return scored
    

In [24]:
def ngram_comparison(corpus, ngram=['bigram', 'trigram', 'quadgram'], threshold=int, top=int, path=None):
    
    """
    Create comparison table for given ngram-type for 
    NLTK association measures based on supplied corpus.
    
    Specify how many top ranking ngrams should be compared. 
    
    Write the ngram table out to the given path if 
    supplied otherwise return the table in memory.
    
    Args:
    corpus (str): corpus that you want to examine
    ngram (str): type of ngram (only one option per function run)
    threshold (int): ignore all ngrams which occur less than n times in the corpus
    top (int): number of top ngrams to be displayed
    path (str): path to save ngram list as .txt file
    
    """
    
    unlist_corpus = [item for items in corpus for item in items]    
    
    if ngram == 'bigram':
        Finder = nltk.collocations.BigramCollocationFinder.from_words(unlist_corpus)
        metric = nltk.collocations.BigramAssocMeasures()
        Finder.apply_freq_filter(threshold)
    
    if ngram == 'trigram':
        Finder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_corpus)
        metric = nltk.collocations.TrigramAssocMeasures()
        Finder.apply_freq_filter(threshold)
        
    if ngram == 'quadgram':
        Finder = nltk.collocations.QuadgramCollocationFinder.from_words(unlist_corpus)
        metric = nltk.collocations.QuadgramAssocMeasures()
        Finder.apply_freq_filter(threshold)

    try:        
        freq_top = pd.DataFrame(list(Finder.score_ngrams(metric.raw_freq)), 
                              columns=['ngram','freq']).sort_values(by='freq', ascending=False)[:top].ngram.values
        
        pmi_top = pd.DataFrame(list(Finder.score_ngrams(metric.pmi)), 
                             columns=['ngram','pmi']).sort_values(by='pmi', ascending=False)[:top].ngram.values
       
        ttest_top = pd.DataFrame(list(Finder.score_ngrams(metric.student_t)), 
                             columns=['ngram','t-test']).sort_values(by='t-test', ascending=False)[:top].ngram.values
       
        chisq_top = pd.DataFrame(list(Finder.score_ngrams(metric.chi_sq)), 
                             columns=['ngram','chisq']).sort_values(by='chisq', ascending=False)[:top].ngram.values
       
        lrt_top = pd.DataFrame(list(Finder.score_ngrams(metric.likelihood_ratio)), 
                             columns=['ngram','lrt']).sort_values(by='lrt', ascending=False)[:top].ngram.values
        
    finally: 
        ngram_comparison = pd.DataFrame([freq_top, pmi_top, ttest_top, chisq_top, lrt_top]).T
        ngram_comparison.columns = ['Frequency', 'PMI', 'T-Test', 'Chi-Square', 'Likelihood Ratio']
    
    if path:
        with open(path, 'w') as f:
            ngram_comparison.to_csv(f, sep='\t', index = False)
            
    else: 
        return ngram_comparison

## 1) Creating & Ranking Ngrams

- Bigrams from cleaned and unprocessed corpus

In [36]:
# raw_freq + minimum 700 occurences + from_words
rank_bigrams(spacy_cleaned, BigramAssocMeasures.raw_freq, from_words=True, threshold=700)

[(('häuslicher', 'gewalt'), 0.004853884071024415),
 (('interventionsstelle', 'häuslicher'), 0.002978918021928341),
 (('bereitschaftsdienst', 'bereitschaftsdienst'), 0.002886922024192319),
 (('telefon', 'telefon'), 0.0027476137990491995),
 (('notfallrettung', 'notarzt'), 0.0026468562777192703),
 (('jugendtelefon', 'nummer'), 0.0026337139923284097),
 (('häusliche', 'gewalt'), 0.002562745651217764),
 (('nummer', 'kummer'), 0.002562745651217764),
 (('ärzteallgemeiner', 'bereitschaftsdienst'), 0.0024944057671852904),
 (('bereitschaftsdienst', 'mi'), 0.0022245508404929583),
 (('mi', 'do'), 0.0018670806778615574),
 (('notarzt', 'jugendtelefon'), 0.0017181347767651403),
 (('sonnabend', 'sonntag'), 0.0015490373714027374),
 (('bereitschaftsdienst', 'do'), 0.0015113628199489378),
 (('do', 'bereitschaftsdienst'), 0.0014736882684951381),
 (('gewalt', 'frauen'), 0.0014307568028849944),
 (('montag', 'freitag'), 0.0014298806505256038),
 (('dienstag', 'donnerstag'), 0.0014018437750251017),
 (('sonntag'

- Trigrams from cleaned and unprocessed corpus

In [32]:
# for cleaned factiva corpus
rank_trigrams(spacy_cleaned, TrigramAssocMeasures.raw_freq, from_words=True, threshold=500,)

[(('jugendtelefon', 'nummer', 'kummer'), 0.0025057957478573692),
 (('notfallrettung', 'notarzt', 'jugendtelefon'), 0.0016524233498108388),
 (('interventionsstelle', 'häuslicher', 'gewalt'), 0.0015464089143245655),
 (('notarzt', 'jugendtelefon', 'nummer'), 0.0014202429745723063),
 (('sonnabend', 'sonntag', 'feiertagen'), 0.0012476409597723405),
 (('gewalt', 'notfallrettung', 'notarzt'), 0.0011854341422556016),
 (('häuslicher', 'gewalt', 'notfallrettung'), 0.0011801772280992575),
 (('entstörungsdienst', 'gas', 'entstörungsdienst'), 0.0010566397454251704),
 (('entstörungsdienst', 'strom', 'wärme'), 0.0010478782218312636),
 (('strom', 'wärme', 'wasser'), 0.0010461259171124823),
 (('häuslicher', 'notfallrettung', 'notarzt'), 0.001038240545877966),
 (('gas', 'entstörungsdienst', 'strom'), 0.001032107479362231),
 (('interventionsstelle', 'häuslicher', 'notfallrettung'),
  0.001026850565205887),
 (('bereitschaftsdienst', 'mi', 'bereitschaftsdienst'), 0.0010224698034089336),
 (('ärzteallgemeine

- Quadgrams from cleaned and unprocessed corpus

In [33]:
# for cleaned factiva corpus
rank_quadgrams(spacy_cleaned, QuadgramAssocMeasures.raw_freq, from_words=True, threshold=700,)

[(('notfallrettung', 'notarzt', 'jugendtelefon', 'nummer'),
  0.0013974630132281482),
 (('notarzt', 'jugendtelefon', 'nummer', 'kummer'), 0.0013659215282900835),
 (('häuslicher', 'gewalt', 'notfallrettung', 'notarzt'),
  0.0011722918568647413),
 (('interventionsstelle', 'häuslicher', 'gewalt', 'notfallrettung'),
  0.00109431429687897),
 (('gas', 'entstörungsdienst', 'strom', 'wärme'), 0.001032107479362231),
 (('entstörungsdienst', 'strom', 'wärme', 'wasser'), 0.0010312313270028405),
 (('entstörungsdienst', 'gas', 'entstörungsdienst', 'strom'),
  0.0010286028699246683),
 (('interventionsstelle', 'häuslicher', 'notfallrettung', 'notarzt'),
  0.0010172128892525895),
 (('kummer', 'entstörungsdienst', 'gas', 'entstörungsdienst'),
  0.000945368395782553),
 (('jugendtelefon', 'nummer', 'kummer', 'entstörungsdienst'),
  0.0009348545674698648),
 (('bereitschaftsdienst', 'do', 'bereitschaftsdienst', 'do'),
  0.0009331022627510834),
 (('mittwoch', 'freitag', 'sonnabend', 'sonntag'), 0.00092959765

## Comparison of association measures for ngrams

- Create bigrams, trigrams and quadgram association measures: top frequency, pmi, t-test, chi-square, likelihood ratio test

In [34]:
# compare assoc measures for bigrams
ngram_comparison(spacy_cleaned, ngram = 'bigram', threshold=3, top=700,)

,Frequency,PMI,T-Test,Chi-Square,Likelihood Ratio
0,"(häuslicher, gewalt)","(anhaben, dickste)","(häuslicher, gewalt)","(abschürfungen, gliedern)","(häuslicher, gewalt)"
1,"(interventionsstelle, häuslicher)","(minimierung, sozialkontakten)","(interventionsstelle, häuslicher)","(republikanisch, gemäßigt)","(notfallrettung, notarzt)"
2,"(bereitschaftsdienst, bereitschaftsdienst)","(schneckentempo, stadiongelände)","(bereitschaftsdienst, bereitschaftsdienst)","(schachttabdeckungen, auszuwechseln)","(nummer, kummer)"
3,"(telefon, telefon)","(schmerzvoll, sehnsüchtige)","(notfallrettung, notarzt)","(sanftmütigem, mütterlich)","(jugendtelefon, nummer)"
4,"(notfallrettung, notarzt)","(schlossbrücke, schaufüttern)","(jugendtelefon, nummer)","(saibling, spreewaldgurken)","(interventionsstelle, häuslicher)"
...,...,...,...,...,...
695,"(weißer, ring)","(buntbemalten, drachen)","(donnerstag, wassergymnastik)","(versorgung, krankheit)","(donnerstag, zusätzlich)"
696,"(sexualität, schwangerschaftskonfliktberatung)","(bratki, kamille)","(montag, trockengymnastik)","(samstags, feiertags)","(telefon, drogenberatung)"
697,"(telefonischer, vereinbarungpalliativnetz)","(blitzsaubere, oberfläche)","(mi, markt)","(lindenplatz, südertor)","(evangelische, kirchgemeinde)"
698,"(telefon, facheinrichtung)","(birke, binse)","(vitae, diezerstraße)","(stationierung, schwedischen)","(samstagärzteallgemeiner, bereitschaftsdienst)"


In [35]:
# compare assoc measures for trigrams
ngram_comparison(spacy_cleaned, ngram = 'trigram', threshold=2, top = 500,)

,Frequency,PMI,T-Test,Chi-Square,Likelihood Ratio
0,"(jugendtelefon, nummer, kummer)","(abtauen, gletscher, wegschmelzen)","(jugendtelefon, nummer, kummer)","(abtauen, gletscher, wegschmelzen)","(interventionsstelle, häuslicher, gewalt)"
1,"(notfallrettung, notarzt, jugendtelefon)","(spielwoche, tischtennisturnier, slackline)","(notfallrettung, notarzt, jugendtelefon)","(spielwoche, tischtennisturnier, slackline)","(jugendtelefon, nummer, kummer)"
2,"(interventionsstelle, häuslicher, gewalt)","(schwerhörigkeit, gehörlosigkeit, tinnitus)","(interventionsstelle, häuslicher, gewalt)","(schwerhörigkeit, gehörlosigkeit, tinnitus)","(notfallrettung, notarzt, jugendtelefon)"
3,"(notarzt, jugendtelefon, nummer)","(schmetterte, making, memories)","(notarzt, jugendtelefon, nummer)","(schmetterte, making, memories)","(notarzt, jugendtelefon, nummer)"
4,"(sonnabend, sonntag, feiertagen)","(sardischen, eisenbiegers, scheffelte)","(sonnabend, sonntag, feiertagen)","(sardischen, eisenbiegers, scheffelte)","(häuslicher, gewalt, notarzt)"
...,...,...,...,...,...
495,"(häusliche, gewalt, stalking)","(gastautorin, preisgekrönte, publizistin)","(häusliche, gewalt, stalking)","(geisterspiele, handballern, basketballern)","(häuslicher, gewalt, verzeichnet)"
496,"(telefon, herzzentrum, notaufnahme)","(fratze, scheußlichkeiten, haaresbreite)","(kontakt, telefon, vitae)","(kommandeur, einsatzführungsbereichs, oberst)","(zweimal, häuslicher, gewalt)"
497,"(kontakt, telefon, vitae)","(flaniermeilen, einkaufszentren, nachtmärkte)","(telefon, herzzentrum, notaufnahme)","(fermate, innehalten, monatsende)","(derzeit, häuslicher, gewalt)"
498,"(strom, gas, notfallrettung)","(feldlerchenweg, neuerschließung, eigenheimsta...","(strom, gas, notfallrettung)","(eckhauses, heruntergezogen, fensterscheiben)","(häuslicher, gewalt, überwiegend)"


In [30]:
# compare assoc measures for quadgrams
ngram_comparison(spacy_cleaned, ngram = 'quadgram', threshold=2, top = 500,)

,Frequency,PMI,T-Test,Chi-Square,Likelihood Ratio
0,"(notfallrettung, notarzt, jugendtelefon, nummer)","(abtauen, gletscher, wegschmelzen, polaren)","(notfallrettung, notarzt, jugendtelefon, nummer)","(abtauen, gletscher, wegschmelzen, polaren)","(notfallrettung, notarzt, jugendtelefon, nummer)"
1,"(notarzt, jugendtelefon, nummer, kummer)","(spezialschuhwerk, außensohle, chromgegerbtem,...","(notarzt, jugendtelefon, nummer, kummer)","(spezialschuhwerk, außensohle, chromgegerbtem,...","(häuslicher, gewalt, notfallrettung, notarzt)"
2,"(häuslicher, gewalt, notfallrettung, notarzt)","(zwischenrufs, bruchstückhaft, wiedergebe, päd...","(häuslicher, gewalt, notfallrettung, notarzt)","(zwischenrufs, bruchstückhaft, wiedergebe, päd...","(notarzt, jugendtelefon, nummer, kummer)"
3,"(interventionsstelle, häuslicher, gewalt, notf...","(winzigsten, stuben, wühltische, bersten)","(interventionsstelle, häuslicher, gewalt, notf...","(winzigsten, stuben, wühltische, bersten)","(interventionsstelle, häuslicher, gewalt, notf..."
4,"(gas, entstörungsdienst, strom, wärme)","(wegschmelzen, polaren, eises, versteppung)","(gas, entstörungsdienst, strom, wärme)","(wegschmelzen, polaren, eises, versteppung)","(interventionsstelle, häuslicher, gewalt, nota..."
5,"(entstörungsdienst, strom, wärme, wasser)","(vindicates, his, stance, fabricated)","(entstörungsdienst, strom, wärme, wasser)","(vindicates, his, stance, fabricated)","(interventionsstelle, häuslicher, notfallrettu..."
6,"(entstörungsdienst, gas, entstörungsdienst, st...","(süßliche, penetrante, leichengeruch, rieche)","(entstörungsdienst, gas, entstörungsdienst, st...","(süßliche, penetrante, leichengeruch, rieche)","(jugendtelefon, nummer, kummer, entstörungsdie..."
7,"(interventionsstelle, häuslicher, notfallrettu...","(stuben, wühltische, bersten, gebrauchtwäsche)","(interventionsstelle, häuslicher, notfallrettu...","(stuben, wühltische, bersten, gebrauchtwäsche)","(interventionsstelle, häuslicher, gewalt, frauen)"
8,"(kummer, entstörungsdienst, gas, entstörungsdi...","(strapazierfähiges, spezialschuhwerk, außensoh...","(kummer, entstörungsdienst, gas, entstörungsdi...","(strapazierfähiges, spezialschuhwerk, außensoh...","(frauen, interventionsstelle, häuslicher, gewalt)"
9,"(jugendtelefon, nummer, kummer, entstörungsdie...","(stilettos, modischen, röcken, stöckeln)","(jugendtelefon, nummer, kummer, entstörungsdie...","(stilettos, modischen, röcken, stöckeln)","(interventionsstelle, häuslicher, gewalt, gasg..."


#### END OF CODE 